In [1]:
import time
import json
import numpy as np
import demjson
import pandas as pd
import urllib
from ipykernel import kernelapp as app

In [2]:
# load data
data_path = "cleaned_us_AI-csv.csv"
unique_inventors = pd.read_csv(data_path)
query_eth_json_base = 'https://api.genderize.io/?'
api_key = '8ee87528e6948e9bf1304ff1c44d4600'

In [3]:
unique_inventors.head()

application_number filing_date  uspc_class  cpc4 inventor_name_first  \
0            12119172  2008-05-12         706  G06N              daniel   
1            14205300  2014-03-11         706  G06N             stephen   
2            12799041  2010-04-16         706  G10L              thomas   
3            13251051  2011-09-30         706  G05B                john   
4            13251051  2011-09-30         706  G05B             michael   

  inventor_name_middle inventor_name_last  inventor_rank inventor_region_code  \
0                    m               rice              1                  NaN   
1                  NaN              lucey              4                  NaN   
2                    d           humphrey              1                   AK   
3               weldon             kelley              2                   AK   
4                    d              scott              5                   AK   

  inventor_country_code  
0                    US  
1                    US  
2                    US  
3                    US  
4                    US

In [5]:
unique_inventors.shape

(21160, 10)

# data clean
unique_names = []
names = set()
for i in range(unique_inventors.shape[0]):
    name = str(unique_inventors.loc[i, 'inventor_first_name']).split()[0]
    name = name.split('.')[0]
    if not name.isalpha() or name in names or len(name) <= 1:
        continue
    else:
        names.add(name)
        unique_names.append(name)

unique = np.array(unique_names)
unique = unique.reshape(len(unique_names), 1)

clean_names = pd.DataFrame(unique,columns=['inventor_first_name'])

In [7]:
# insert new colomns first
genders = ['gender', 'probability', 'count']
i = 10
for gen in genders:
    new_col = pd.Series([0]*unique_inventors.shape[0])
    unique_inventors.insert(i, gen, new_col)
    i += 1

In [9]:
unique_inventors.head()

application_number filing_date  uspc_class  cpc4 inventor_name_first  \
0            12119172  2008-05-12         706  G06N              daniel   
1            14205300  2014-03-11         706  G06N             stephen   
2            12799041  2010-04-16         706  G10L              thomas   
3            13251051  2011-09-30         706  G05B                john   
4            13251051  2011-09-30         706  G05B             michael   

  inventor_name_middle inventor_name_last  inventor_rank inventor_region_code  \
0                    m               rice              1                  NaN   
1                  NaN              lucey              4                  NaN   
2                    d           humphrey              1                   AK   
3               weldon             kelley              2                   AK   
4                    d              scott              5                   AK   

  inventor_country_code  gender  probability  count  
0                    US       0            0      0  
1                    US       0            0      0  
2                    US       0            0      0  
3                    US       0            0      0  
4                    US       0            0      0

In [10]:
# query for gender
cnt = 0
for i in range(unique_inventors.shape[0]):
    cnt += 1
    if cnt%100 == 0:
        print(cnt)
    f_name = str(unique_inventors.loc[i, 'inventor_name_first'])
    m_name = str(unique_inventors.loc[i, 'inventor_name_middle'])
    #country = unique_inventors.loc[i, 'inventor_country']
    if len(f_name) < 2 and len(m_name) < 2:
        unique_inventors.loc[i, 'valid'] = 'N'
        continue
    if len(f_name) < 2:
        name = m_name
    else:
        name = f_name
    
    name = urllib.request.quote(name)
    url = query_eth_json_base + 'name=' + name + '&'
    url += 'apikey=' + api_key
    try:
        response = urllib.request.urlopen(url).read()
    except:
        print(i, url)
        unique_inventors.loc[i, 'valid'] = 'N'
    else:
        response = response.decode("utf-8") 
        result = json.loads(response)
        for eth in result:
            unique_inventors.loc[i, eth] = result[eth]

100
200
300
400
500
600
700
800
900
1000
1100
1146 https://api.genderize.io/?name=john&apikey=8ee87528e6948e9bf1304ff1c44d4600
1200
1300
1400
1408 https://api.genderize.io/?name=robert&apikey=8ee87528e6948e9bf1304ff1c44d4600
1500
1600
1700
1800
1900
2000
2100
2200
2213 https://api.genderize.io/?name=anugeetha&apikey=8ee87528e6948e9bf1304ff1c44d4600
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5359 https://api.genderize.io/?name=eric&apikey=8ee87528e6948e9bf1304ff1c44d4600
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7684 https://api.genderize.io/?name=rony&apikey=8ee87528e6948e9bf1304ff1c44d4600
7700
7800
7900
8000
8100
8200
8300
8342 https://api.genderize.io/?name=eric&apikey=8ee87528e6948e9bf1304ff1c44d4600
8400
8500
8600
8700
8800
8900
9000
9088 https://api.genderize.io/?name=evangelos&apikey=8ee87528e6948

In [12]:
unique_inventors.to_csv("cleaned_us_AI_gender.csv")